In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [2]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from IPython.display import clear_output
tf= TfidfVectorizer()

In [3]:
train_df= pd.read_pickle('train.pickle')
X= train_df['clean_lemma'][:200]
Y= train_df['target'][:200]
test_df= pd.read_pickle('test.pickle')
print(train_df.shape)
print(test_df.shape)
display(train_df.head())
test_df.head()

(7613, 3)
(3263, 2)


,id,clean_lemma,target
0,1,allah deed reason forgive,1
1,4,forest ronge sask canada fire la near,1
2,5,resident officer place order ask notify expect...,1
3,6,order california people evacuation receive,1
4,7,school photo ruby smoke send get pour,1


,id,text
0,0,just happened a terrible car crash
1,2,heard about is different cities stay safe ever...
2,3,there is a forest fire at spot pond geese are ...
3,9,apocalypse lighting
4,11,typhoon soudelor kills in china and taiwan


In [4]:
train= tf.fit_transform(X).todense()
print(train.shape[0],'Examples with',train.shape[1],'features')

200 Examples with 843 features


In [5]:
from sklearn.model_selection import train_test_split
train_x,val_x,train_y,val_y= train_test_split(train,Y)

In [6]:
train_x.shape

(150, 843)


##  Implementing 2 Layer network with 1 Hidden Layer

In [6]:

class NeuralNetwork_2layer_1H:
    def __init__(self,num_iters,lr):
        self.lr= lr
        
        self.W1= None
        self.DW1= None

        self.W2= None
        self.DW2= None
        
        self.B1= None
        self.DB1= None
        
        self.B2= None
        self.DB2= None
        
        self.DZ1= None
        self.DZ2= None
        
        self.num_iters= num_iters
        self.N= None
        
        self.X= None
        self.Y= None
        self.A2=None
    def initialize_parameters(self,X,Y,nodes):
        self.X= X
        self.Y= Y
        self.W1= np.random.randint(5,size= (nodes,X.shape[1])) *0.01
        self.B1= np.zeros((nodes,1))
        self.W2= np.random.randint(5,size=((1,nodes))) *0.01
        self.B2= np.zeros((1,1))
        self.N= X.shape[0]        

    
    def Print_parameters(self):
        print('Initializing parameters..')
        print('W1:',self.W1.shape,'DW1:',self.DW1.shape)
        
        print('B1:',self.B1.shape,'DB1:',self.DB1.shape)
        
        
        print('W2:',self.W2.shape,'DW2:',self.DW2.shape)
        
        print('B2:',self.B2.shape,'DB2',self.DB2.shape)
        
        print('Z1:',self.Z1.shape,'DZ1:',self.DZ1.shape)
        
        print('Z2:',self.Z2.shape,'DZ2:',self.DZ2.shape)
        print()
        
    def sigmoid(self,z):
        return 1/(1+np.exp(-z))
    
    def feed_forward(self):
        self.Z1= np.dot(self.W1,self.X.T)+self.B1

        A1= np.tanh(self.Z1)

        self.Z2= np.dot(self.W2,A1)+self.B2
        
        self.A2= self.sigmoid(self.Z2)

        self.DZ2= self.A2 -self.Y
        
        self.DW2= (1/self.N)*np.dot(self.DZ2,A1.T)
        self.DB2= (1/self.N)*np.sum(self.DZ2,axis=1)
        
        d_act= np.multiply(self.Z1 , (1-self.Z1))
        self.DZ1= (1/self.N)*np.multiply(np.dot(self.W2.T,self.DZ2), d_act)
        
        self.DW1= (1/self.N)*np.dot(self.DZ1,self.X)
        self.DB1= (1/self.N)*np.sum(self.DZ1,axis=1)
    
    def find_cost(self):
        a= np.ravel(np.dot(self.Y , np.log(self.A2).T))
        b= np.ravel(np.dot((1-self.Y),np.log(1-self.A2.T)))
        cost = -(1/self.N) * (a+b)
        print('Cost :',cost)

        
        
    def Gradient_descent(self):
        for i in range(self.num_iters):
            self.feed_forward()
            print('Iteration:',i)
            self.find_cost()
            self.W1= self.W1 - self.lr * self.DW1
            self.B1= self.B1 - self.lr * self.DB1
            self.W2= self.W2 - self.lr * self.DW2
            self.B2= self.B2 - self.lr * self.DB2
            clear_output(wait=True)
            
    
    def fit(self,X,Y,nodes):
        self.initialize_parameters(X,Y,nodes)
        self.Gradient_descent()
        self.find_cost()
        print('Ran Successfully')
    


num_iters=1000
learning_rate=4.5
hidden_nodes= 5

nn= NeuralNetwork_2layer_1H(num_iters,learning_rate)
nn.fit(train_x,np.array([train_y]),hidden_nodes)

Cost : [0.04897911]
Ran Successfully


## Implementing N layer Neural Network using  relu function in Hidden layers and Sigmoid in Output layer

In [7]:
train_x= train_x.T

In [8]:
%xmode plain
class Deep_Neural_network:
    def __init__(self):
        self.X= None
        self.Y= None
        self.layers= None
        self.parameters= {'W':{},'B':{},'A':{},'Z':{}}
        self.gradients= {'DW':{},'DB':{},'DA':{},'DZ':{}}
        self.N= None
        self.L= None
        self.num_iteration= None
    def printing_func(self):
        print('Layers :',self.layers)
        print('X :',self.X.shape)
        print('Y :',self.Y.shape)
        print('L :',self.L)
        print('W :',[(w,self.parameters['W'][w].shape) for w in self.parameters['W'].keys()])
        print('B :',[(b,self.parameters['B'][b].shape) for b in self.parameters['B'].keys()])
        print('Z :',[(z,self.parameters['Z'][z].shape) for z in self.parameters['Z'].keys()])
        print('A :',[(a,self.parameters['A'][a].shape) for a in self.parameters['A'].keys()])
        print('DZ :',[(dz,self.gradients['DZ'][dz].shape) for dz in self.gradients['DZ'].keys()])
        print('DA :',[(da,self.gradients['DA'][da].shape) for da in self.gradients['DA'].keys()])
        print('DW :',[(dw,self.gradients['DW'][dw].shape) for dw in self.gradients['DW'].keys()])
        
    def initialize_parameters(self,X,Y,layers,num_iteration):
        layers.insert(0,X.shape[0])
        layers.append(1)
        self.num_iteration= num_iteration
        self.X= X
        self.lr= 0.001
        self.Y= Y
        self.layers= layers
        self.L= len(self.layers)
        self.N= len(X)
        self.parameters['A']['A0']= X
        
        for l in range(1,self.L):
            self.parameters['W']['W'+str(l)]= np.random.randint(low=1,high=9,size= (self.layers[l],self.layers[l-1]))*0.01
            self.parameters['B']['B'+str(l)]= np.zeros((self.layers[l],1))
        
            
    def linear_forward(sekf,W,X,B):
        Z= np.dot(W,X)+B
        return Z 
    
    def relu(self,Z):
        z= np.maximum(0,Z)
        
        return z
    
    def sigmoid(self,Z):
        z= 1/(1+np.exp(-Z))
        return z
    
    def forward_pass(self):
        for l in range(1,self.L):
            W= self.parameters['W']['W'+str(l)]
            B= self.parameters['B']['B'+str(l)]
            X= self.parameters['A']['A'+str(l-1)]
            self.parameters['Z']['Z'+str(l)]= self.linear_forward(W,X,B)
            
            if l == self.L-1:
                self.parameters['A']['A'+str(l)]= self.sigmoid(self.parameters['Z']['Z'+str(l)])
            else:
                self.parameters['A']['A'+str(l)]= self.relu(self.parameters['Z']['Z'+str(l)])
                
    def relu_backward(self,Z):
        Z[Z<=0]=0
        return Z
    
    def cost(self):
        Y_pred= self.parameters['A']['A'+str(self.L-1)]
        
        a= np.ravel(np.dot(self.Y , np.log(Y_pred).T))
        b= np.ravel(np.dot((1-self.Y),np.log(Y_pred).T))
        
        cost = -(1/self.N) * (a+b)
        print('Cost :',cost)
        return cost

    def sigmoid_backward(self,Z):
        Z= np.multiply(Z,(1-Z))
        return Z
    
    def backward_pass(self):
        self.gradients['DA']['DA'+str(self.L-1)]= - np.subtract( np.divide(self.Y,self.parameters['A']['A'+str(self.L-1)]) ,
        np.divide(1- self.Y,1- self.parameters['A']['A'+str(self.L-1)]))
        for l in reversed(range(1,self.L)):
            if l == self.L-1:
                self.gradients['DZ']['DZ'+str(l)]= np.multiply(self.gradients['DA']['DA'+str(l)],self.sigmoid(self.parameters['Z']['Z'+str(l)]))
                assert(self.gradients['DZ']['DZ'+str(l)].shape == self.parameters['Z']['Z'+str(l)].shape)
            else:
                self.gradients['DZ']['DZ'+str(l)]= np.multiply(self.gradients['DA']['DA'+str(l)],self.relu_backward(self.parameters['Z']['Z'+str(l)]))
                assert(self.gradients['DZ']['DZ'+str(l)].shape == self.parameters['Z']['Z'+str(l)].shape)

            self.gradients['DW']['DW'+str(l)]= np.dot(self.gradients['DZ']['DZ'+str(l)],self.parameters['A']['A'+str(l-1)].T) * (1/self.N)
            assert(self.gradients['DW']['DW'+str(l)].shape== self.parameters['W']['W'+str(l)].shape)

            self.gradients['DB']['DB'+str(l)]= np.sum(self.gradients['DZ']['DZ'+str(l)],axis=1) *(1/self.N)
            assert(self.gradients['DB']['DB'+str(l)].shape==self.parameters['B']['B'+str(l)].shape)

            if l==1:
                continue 

            self.gradients['DA']['DA'+str(l-1)]= np.dot(self.parameters['W']['W'+str(l)].T,self.gradients['DZ']['DZ'+str(l)])
            assert(self.gradients['DA']['DA'+str(l-1)].shape== self.parameters['A']['A'+str(l-1)].shape)

    def gradient_descent(self):
        for x in range(1000):
            self.forward_pass()
            self.backward_pass()
            for g in range(1,self.L):
                self.parameters['W']['W'+str(g)]= self.parameters['W']['W'+str(g)]- (self.lr * self.gradients['DW']['DW'+str(g)])
                self.parameters['B']['B'+str(g)]= self.parameters['B']['B'+str(g)]- (self.lr * self.gradients['DB']['DB'+str(g)])
            if x%100==0:
                self.cost()

    def fit(self,X,Y,layers,num_iteration):
        self.initialize_parameters(X,Y,layers,num_iteration)
        self.gradient_descent()
        self.printing_func()


        
np.random.seed() 
num_hidden_layer=3
num_iteration=1000
layers=list(np.random.randint(low=2,high=5,size= num_hidden_layer))
dnn= Deep_Neural_network()
dnn.fit(train_x,np.array([train_y]),layers,num_iteration)

Exception reporting mode: Plain
Cost : [0.12330948]
Cost : [0.12330969]
Cost : [0.1233099]
Cost : [0.12331011]
Cost : [0.12331031]
Cost : [0.12331052]
Cost : [0.12331072]
Cost : [0.12331092]
Cost : [0.12331112]
Cost : [0.12331132]
Layers : [843, 3, 4, 4, 1]
X : (843, 150)
Y : (1, 150)
L : 5
W : [('W1', (3, 843)), ('W2', (4, 3)), ('W3', (4, 4)), ('W4', (1, 4))]
B : [('B1', (3, 1)), ('B2', (4, 1)), ('B3', (4, 1)), ('B4', (1, 1))]
Z : [('Z1', (3, 150)), ('Z2', (4, 150)), ('Z3', (4, 150)), ('Z4', (1, 150))]
A : [('A0', (843, 150)), ('A1', (3, 150)), ('A2', (4, 150)), ('A3', (4, 150)), ('A4', (1, 150))]
DZ : [('DZ4', (1, 150)), ('DZ3', (4, 150)), ('DZ2', (4, 150)), ('DZ1', (3, 150))]
DA : [('DA4', (1, 150)), ('DA3', (4, 150)), ('DA2', (4, 150)), ('DA1', (3, 150))]
DW : [('DW4', (1, 4)), ('DW3', (4, 4)), ('DW2', (4, 3)), ('DW1', (3, 843))]


In [9]:
dnn.gradients

{'DW': {'DW4': matrix([[-2.19008820e-05, -3.77035937e-05, -3.20449870e-05,
           -2.76386202e-05]]),
  'DW3': matrix([[-8.83636998e-09, -3.73150248e-08, -2.39841380e-08,
           -1.85334911e-08],
          [-3.05935747e-09, -1.29195260e-08, -8.30381246e-09,
           -6.41758687e-09],
          [-1.04337303e-08, -4.40687099e-08, -2.83216202e-08,
           -2.18992160e-08],
          [-8.93889751e-09, -3.77505240e-08, -2.42632166e-08,
           -1.87521783e-08]]),
  'DW2': matrix([[-3.44263990e-10, -2.69830331e-10, -2.35301371e-10],
          [-1.40034313e-09, -1.09482925e-09, -9.57606536e-10],
          [-8.12374007e-10, -6.35450185e-10, -5.55232478e-10],
          [-6.10340960e-10, -4.77240984e-10, -4.19021045e-10]]),
  'DW1': matrix([[-6.39105087e-13, -7.77842495e-13,  0.00000000e+00, ...,
            1.44545256e-12,  6.96399689e-13, -1.69601790e-12],
          [-1.71320379e-12, -1.26064795e-12,  0.00000000e+00, ...,
            2.05333755e-12,  1.21124802e-12, -2.88855292